In [33]:
import torch
from models.CLIP import Adapter

In [34]:
# torch.load adapter
adapter1 = Adapter(c_in=512, reduction=4, bias=True)
adapter1.load_state_dict(torch.load('../weights/0_adapter.pt'))

adapter2 = Adapter(c_in=512, reduction=4, bias=True)
adapter2.load_state_dict(torch.load('../weights/2_adapter.pt'))

<All keys matched successfully>

In [35]:
adapter1.state_dict()

OrderedDict([('fc.0.weight',
              tensor([[ 0.0002,  0.0500, -0.0150,  ..., -0.0270,  0.0182, -0.0579],
                      [-0.0116,  0.0345,  0.0159,  ...,  0.0178,  0.0372, -0.0225],
                      [ 0.0153, -0.0211,  0.0166,  ...,  0.0184,  0.0561,  0.0131],
                      ...,
                      [ 0.0116, -0.0066,  0.0131,  ...,  0.0244, -0.0202, -0.0495],
                      [ 0.0087, -0.0035, -0.0055,  ..., -0.0357, -0.0120, -0.0481],
                      [-0.0139,  0.0197, -0.0367,  ..., -0.0039,  0.0133,  0.0216]])),
             ('fc.0.bias',
              tensor([-0.0133, -0.0174, -0.0013, -0.0231,  0.0249, -0.0098, -0.0182, -0.0042,
                       0.0066, -0.0088,  0.0281,  0.0308,  0.0307,  0.0094, -0.0186,  0.0298,
                      -0.0103,  0.0399, -0.0041, -0.0073,  0.0413, -0.0033,  0.0059,  0.0079,
                       0.0285, -0.0041,  0.0022,  0.0124,  0.0093,  0.0312,  0.0189,  0.0058,
                      -0.0068,  0.

In [36]:
adapter2.state_dict()

OrderedDict([('fc.0.weight',
              tensor([[-0.0389,  0.1063, -0.0929,  ..., -0.0370,  0.0082, -0.0721],
                      [ 0.0279,  0.0304, -0.0660,  ...,  0.0687, -0.0152, -0.0128],
                      [-0.0015, -0.0368,  0.0412,  ..., -0.0819,  0.0825, -0.0127],
                      ...,
                      [ 0.0055,  0.0359, -0.0159,  ...,  0.0290, -0.0185,  0.0040],
                      [ 0.0556, -0.0552, -0.1014,  ..., -0.1625, -0.0114, -0.0236],
                      [ 0.0427,  0.0605, -0.0414,  ..., -0.0442, -0.0170, -0.0180]])),
             ('fc.0.bias',
              tensor([ 0.0036, -0.0354, -0.0101, -0.0096, -0.0028, -0.0245, -0.0252, -0.0019,
                       0.0123,  0.0079,  0.0162,  0.0159,  0.0242,  0.0068, -0.0288,  0.0245,
                      -0.0214,  0.0327, -0.0221, -0.0073,  0.0158, -0.0017, -0.0024, -0.0001,
                       0.0283, -0.0084,  0.0022,  0.0155,  0.0038,  0.0114, -0.0023, -0.0138,
                      -0.0151,  0.

In [37]:
for (name1, param1), (name2, param2) in zip(adapter1.named_parameters(), adapter2.named_parameters()):
    # norm diff
    diff = torch.norm(param1 - param2).item()

    # cal cosine similarity
    cos = torch.nn.functional.cosine_similarity(param1.flatten(), param2.flatten(), dim=0)

    print(f'{name1} vs {name2}: diff={diff}, cos={cos}')

fc.0.weight vs fc.0.weight: diff=13.572736740112305, cos=0.2932654023170471
fc.0.bias vs fc.0.bias: diff=0.1520899534225464, cos=0.7863556742668152
fc.2.weight vs fc.2.weight: diff=10.942910194396973, cos=0.43735986948013306
fc.2.bias vs fc.2.bias: diff=0.2769106924533844, cos=0.8472673892974854


In [40]:
import torch
import torch.nn as nn
import copy
x1 = torch.arange(6).view(2,-1).type(torch.float64)
g1 = nn.Linear(3,2, bias=False).type(torch.float64)
g2 = nn.Linear(2,3, bias=False).type(torch.float64)
l1 = nn.Linear(3,2, bias=False).type(torch.float64)
l2 = nn.Linear(2,3, bias=False).type(torch.float64)

w1 = torch.randn(2,2).type(torch.float64)
w2 = torch.randn(3,3).type(torch.float64)

G1 = g1(x1) @ w1
L1 = l1(x1) @ w1
G1 = G1 + L1
G2 = g2(G1) @ w2
L2 = l2(G1) @ w2

g1_ = copy.deepcopy(g1)
g2_ = copy.deepcopy(g2)
l1_ = copy.deepcopy(l1)
l2_ = copy.deepcopy(l2)

g1_.weight.data = w1.t() @ g1.weight.data.clone()
g2_.weight.data = w2.t() @ g2.weight.data.clone()
l1_.weight.data = w1.t() @ l1.weight.data.clone()
l2_.weight.data = w2.t() @ l2.weight.data.clone()

G1_ = g1_(x1)
L1_ = l1_(x1)
G1_ = G1_ + L1_
G2_ = g2_(G1)
L2_ = l2_(G1)

print(torch.equal(G1, G1_))

True


In [41]:
torch.equal(g1(x1) @ w1 , x1 @ g1.weight.data.t() @ w1)

True

In [42]:
g1(x1) @ w1

tensor([[-0.4110,  0.6556],
        [-1.0680,  1.7786]], dtype=torch.float64, grad_fn=<MmBackward0>)

In [43]:
x1 @ g1.weight.data.t() @ w1

tensor([[-0.4110,  0.6556],
        [-1.0680,  1.7786]], dtype=torch.float64)

In [44]:
x1 @ (w1.T @ g1.weight.data).T

tensor([[-0.4110,  0.6556],
        [-1.0680,  1.7786]], dtype=torch.float64)

In [45]:
x1 @ (g1.weight.data.t() @ w1) - x1 @ (w1.T @ g1.weight.data).T

tensor([[0., 0.],
        [0., 0.]], dtype=torch.float64)

In [46]:
torch.equal(g1(x1) , x1 @ g1.weight.data.t())

True

In [47]:
torch.equal(g1(x1) @ w1 , x1 @ (g1.weight.data.t() @ w1))

True

In [48]:
torch.equal(x1 @ g1.weight.data.t() @ w1 , x1 @ (g1.weight.data.t() @ w1))

True

In [49]:
torch.equal(x1 @ (w1.T @ g1.weight.data).T , x1 @ (g1.weight.data.T @ w1))

True

In [50]:
torch.equal(g1(x1) @ w1, x1 @ (w1.T @ g1.weight.data).T)

True